<a href="https://colab.research.google.com/github/kunakl07/Killer-Whale-sound-classification/blob/master/1_Whale_sound_detection_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/axiom-data-science/OrcaCNN-data

Cloning into 'OrcaCNN-data'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 68 (delta 3), reused 61 (delta 2), pack-reused 0
Unpacking objects: 100% (68/68), done.


In [0]:
cd /content/OrcaCNN-data

/content/OrcaCNN-data


In [0]:
rm /content/OrcaCNN-data/data/training/positive/class_labels.json

In [0]:
!pwd

/content/OrcaCNN-data


In [0]:

import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

DATA_PATH = "/content/OrcaCNN-data/data/training/"


# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


# Handy function to convert wav2mfcc
def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc


def save_data_to_array(path=DATA_PATH, max_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)


def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)



def prepare_dataset(path=DATA_PATH):
    labels, _, _ = get_labels(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path  + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
            # Downsampling
            wave = wave[::3]
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data


def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]


# print(prepare_dataset(DATA_PATH))



Using TensorFlow backend.


In [0]:



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 500
batch_size = 100
verbose = 1
num_classes = 2

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)


Saving vectors of label - 'negative': 100%|██████████| 25/25 [00:01<00:00, 15.97it/s]


In [0]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]


In [0]:

model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 26 samples, validate on 18 samples
Epoch 1/1200
26/26 [==============================] - 5s 200ms/step - loss: 1.3004 - acc: 0.5000 - val_loss: 2.3386 - val_acc: 0.5556
Epoch 2/1200
26/26 [==============================] - 0s 731us/step - loss: 1.8339 - acc: 0.5769 - val_loss: 2.5927 - val_acc: 0.4444
Epoch 3/1200
26/26 [==============================] - 0s 559us/step - loss: 2.5375 - acc: 0.4615 - val_loss: 3.0166 - val_acc: 0.5556
Epoch 4/1200
26/26 [==============================] - 0s 575us/step - loss: 3.1144 - acc: 0.5769 - val_loss: 0.6533 - val_acc: 0.6667
Epoch 5/1200
26/26 [==============================] - 0s 510us/step - loss: 0.7982 - acc: 0.7308 - val_loss: 0.6702 - val_acc: 0.5556
Epoch 6/1200
26/26 [==============================]

In [0]:
!pip install pydub


In [0]:
  from pydub import AudioSegment

AUDIO_FILE="/content/OrcaCNN-data/data/long_samples/long_sample_01.wav"
sound = AudioSegment.from_file(AUDIO_FILE)
i=0
k=0
p=0
m=len(sound)
halfway_point = 4000
for i in range(i,m,4000):

        first_half = sound[p:halfway_point]
     #   sh=sound[p+4000:halfway_point+4000]


        k=k+4000
        first_half.export("/content/OrcaCNN-data/data/long_sample{0}.wav".format(k),format="wav")
    #    sh.export("/content/Pests-data/popo{0}.wav".format(k),format="wav")
        halfway_point=halfway_point+4000
        p=p+4000

In [0]:
l=4000
i=4000
u=0
for i in range(i,m,4000):
  u=u+1
  print("For sample{0} the output is".format(u))
  print(predict('/content/OrcaCNN-data/data/long_sample{0}.wav'.format(l), model=model))
  l=l+4000

For sample1 the output is
positive
For sample2 the output is
positive
For sample3 the output is
negative
For sample4 the output is
positive
For sample5 the output is
positive
For sample6 the output is
negative
For sample7 the output is
negative


In [0]:
'''
import os
import wave

import pylab
def graph_spectrogram(wav_file):
    sound_info, frame_rate = get_wav_info(wav_file)
    pylab.figure(num=None, figsize=(19, 12))
    pylab.subplot(111)
    pylab.title('spectrogram of %r' % wav_file)
    pylab.specgram(sound_info, Fs=frame_rate)
    pylab.savefig('spectrogram.png')
def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.fromstring(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate
graph_spectrogram('/content/OrcaCNN-data/data/long_samples/long_sample_01.wav')'''

In [0]:
!pip install -U numpy==1.11.0.


In [0]:
!pip install numpy==1.11.0